In [5]:

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader,PyPDFDirectoryLoader
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

In [ ]:
#intialize embedings
embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [26]:
#load or create vectordb

try:
    vectorestore=FAISS.load_local('faiss_index',embeddings,allow_dangerous_deserialization=True)
    print('loadded existind vectordb')
except:
    #load pdf
    loader=PyPDFDirectoryLoader('data')
    docs=loader.load()

    #split into chunk
    text_spliter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
    splits=text_spliter.split_documents(docs)

    #create and save vector db
    vectorestore=FAISS.from_documents(splits,embeddings)
    vectorestore.save_local('faiss_index')
    print('created and saved vector db')

created and saved vector db


In [27]:
#create retriever from vectordb

retriever=vectorestore.as_retriever(search_kwargs={'k':5})


In [28]:
#initialize ollama llm
llm=Ollama(model='llama3')

In [29]:
#qa retrieval chain

qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever
    #return_source_documents=True
)

## test

In [ ]:
query='what is the fox'
result=qa_chain({'query':query})
print('Answer:',result['result'])